In [141]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
import os
import io

In [5]:
def leer_pdf(file) -> str:
    """
    Lee un archivo PDF usando LangChain y extrae su contenido como texto.

    Args:
        file (UploadFile): Archivo PDF cargado.

    Returns:
        str: Texto extraído del PDF.
    """
    try:
        loader = PyPDFLoader(file)
        
        # Cargar el documento
        documento = loader.load()
        return documento

    except Exception as e:
        raise ValueError(f"Error al leer el archivo PDF: {str(e)}")


In [171]:
# Configuración de RAG
def configurar_rag(documento, puesto) -> RetrievalQA:
    """
    Configura la cadena RAG para el modelo de optimización de CV.

    Args:
        documentos (list): Lista de textos de referencia para el almacenamiento vectorial.

    Returns:
        RetrievalQA: Cadena RAG configurada.
    """
    # Configurar embeddings y vector store
    documento = leer_pdf(documento)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documento)
    
    # Crear embeddings
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_store = FAISS.from_documents(docs, embeddings)
    
    # Configurar el modelo de lenguaje y el prompt
    prompt_template =f"""
            "Lee el siguiente currículum y extrae las partes más importantes relacionadas con la consulta del usuario. Proporciona detalles clave como la experiencia laboral, habilidades técnicas, formación académica, logros, certificaciones y cualquier otra información relevante que responda directamente a la consulta. Asegúrate de que la información extraída sea específica y organizada, y destaca las secciones que se ajusten mejor a lo que está buscando el usuario.

            Consulta del usuario: {puesto}


            Responde con la información más relevante relacionada con la consulta que debería estar relacionada al puesto de trabajo al que aspira el usuario. ten en cuenta que tienes un máximo de 1000 tokens para responder"
    """
    # Crear la cadena RAG
    llm = ChatGroq(model_name="mixtral-8x7b-32768", groq_api_key="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", temperature= 0, max_tokens=1000)
    rag_chain = RetrievalQA.from_chain_type(
        retriever=vector_store.as_retriever(),
        return_source_documents=True,
        llm=llm
    )
    response = rag_chain.invoke(prompt_template)
    return response['result']
configurar_rag(ruta, "Analista de Datos")

"Santiago García Martínez is an Economics student with a solid understanding of economic principles and a natural talent for leading teams towards their goals with resilience and determination. He has experience in financial risk management and is skilled in using various statistical analysis tools such as Excel, Python, SQL, PowerBi, Stata, R, Microsoft Office, and Siigo. Santiago has provided personalized advisory and machine learning services from 2023-present in Cali. He developed economic and financial problem-solving models using tools like Python, SQL, scikit-learn, statsmodels, nltk, linearmodels, and R, resulting in improved predictive accuracy in economic forecasting and text non-structured data extraction.\n\nSantiago has experience as a financial assistant at Quad Ingeniería from 2019-2022 in Cali, where he created projected cash flows, developed formats in various tools, managed the company's CRM, and created promotional flyers for the company's events. He learned basic Il

In [177]:
# Definir un agente que reciba la salida de la herramienta RAG y genere una respuesta en español
def configurar_agente_rag(documento, puesto):
    """
    Configura el agente LangChain que toma la respuesta de la herramienta RAG y genera una respuesta optimizada en español.

    Args:
        documento (str): Ruta del documento PDF.
        puesto (str): Nombre del puesto de trabajo objetivo.
    
    Returns:
        str: Respuesta generada por el agente en español.
    """
    # Obtener la respuesta de la herramienta RAG
    resultado_rag = configurar_rag(documento, puesto)
    
    # Definir un prompt para el agente LangChain
    prompt_template = f"""
    Rol: Eres un asistente virtual especializado en optimización de currículums, utilizando técnicas avanzadas para mejorar la presentación y alineación del perfil del candidato con los requisitos de un puesto específico. Tu tarea es analizar el currículum proporcionado por el usuario para el puesto de trabajo {puesto} y ofrecer recomendaciones detalladas para optimizar el perfil del candidato.

    Información proporcionada:
    
    Análisis del currículum: {resultado_rag}
    Puesto de trabajo: {puesto}
    Instrucciones:
    
    Revisa cuidadosamente el currículum y compáralo con los requisitos del puesto proporcionado.
    Utiliza técnicas de optimización de currículums para mejorar la efectividad del perfil del candidato. Algunas técnicas incluyen:
    Resaltar logros cuantificables.
    Asegurar la inclusión de palabras clave relevantes para el puesto.
    Optimizar la estructura para facilitar la lectura y destacar las secciones más importantes.
    Usar un formato atractivo y profesional que facilite el escaneo rápido por los reclutadores.
    Proporciona recomendaciones específicas para mejorar el currículum, basándote en las áreas que necesitan mayor ajuste para alinearse mejor con el puesto de trabajo.
    Haz énfasis en los siguientes aspectos:
    Experiencia laboral: Sugiere cómo resaltar logros clave o responsabilidades relevantes, utilizando verbos de acción y cifras cuantificables.
    Habilidades técnicas: Identifica habilidades faltantes que podrían ser incluidas, si el candidato las posee, y cómo presentarlas de manera efectiva.
    Educación y formación: Señala cómo resaltar títulos o certificaciones relevantes para el puesto, destacando cualquier formación adicional pertinente.
    Logros y competencias adicionales: Recomendaciones sobre cómo mejorar la sección de logros y habilidades blandas, utilizando ejemplos concretos cuando sea posible.
    Personaliza la respuesta si el candidato menciona su nombre, utilizando un tono amigable y motivacional.
    Objetivo: Ayudar al candidato a mejorar su currículum para aumentar sus posibilidades de éxito en la postulación al puesto de {puesto}, utilizando técnicas de optimización de currículums que lo hagan destacar entre otros postulantes."""



    # Crear un modelo de lenguaje de LangChain 
    llm = ChatGroq(model="gemma2-9b-it", groq_api_key="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", max_tokens=1000) 

    # Crear un LLMChain con el prompt y el modelo
    llm_chain = LLMChain(prompt=PromptTemplate.from_template(prompt_template), llm=llm)

    # Ejecutar el agente para generar la respuesta
    respuesta = llm_chain.run(prompt= puesto)
    
    return respuesta

# Ejemplo de uso
respuesta = configurar_agente_rag(ruta, "Analista de Datos")
print(respuesta)

¡Hola Santiago! 

Analicé tu currículum y veo que tienes una sólida base para ser un Analista de Datos exitoso. Tu experiencia en análisis financiero, manejo de bases de datos y dominio de herramientas como Python, SQL y R son puntos fuertes que se alinean perfectamente con los requisitos del puesto. 

Para que tu currículum destaque aún más, te recomiendo lo siguiente:

**Experiencia Laboral:**

* **Cuantifica tus logros:** En tu puesto de Finanzas en Quad Ingeniería, en lugar de simplemente mencionar "creé proyecciones de flujo de caja" o "realicé análisis financieros", agrega datos específicos. Por ejemplo: "Creé proyecciones de flujo de caja que **redujeron el error de pronóstico en un 15%**" o "Realicé análisis financieros que **identificaron una oportunidad de ahorro de $X en costos operativos**".
* **Resalta responsabilidades relevantes:** Destaca responsabilidades que demuestren tu capacidad para analizar datos y tomar decisiones basadas en ellos. Por ejemplo, si utilizaste dat